In [0]:
# Drop existing silver table if it exists
spark.sql("DROP TABLE IF EXISTS silver_db.jobs_silver")

# Optionally, drop silver database too
# spark.sql("DROP DATABASE IF EXISTS silver_db CASCADE")


DataFrame[]

In [0]:
# Read the bronze table from Delta Lake
bronze_df = spark.read.table("bronze_db.jobs_bronze")


bronze_df.printSchema()
print(f"Bronze Row Count: {bronze_df.count()}")


root
 |-- job_id: long (nullable = true)
 |-- title_raw: string (nullable = true)
 |-- company: string (nullable = true)
 |-- location_raw: string (nullable = true)
 |-- post_date: string (nullable = true)
 |-- url: string (nullable = true)
 |-- type_raw: string (nullable = true)
 |-- levels_raw: string (nullable = true)
 |-- categories_raw: string (nullable = true)
 |-- contents: string (nullable = true)
 |-- _ingest_ts: string (nullable = true)
 |-- _run_label: string (nullable = true)
 |-- _source: string (nullable = true)

Bronze Row Count: 1212


In [0]:
from pyspark.sql.functions import col, count, when

bronze_df.select([
    count(when(col(c).isNull(), c)).alias(c) for c in bronze_df.columns
]).show(vertical=True)


-RECORD 0-------------
 job_id         | 0   
 title_raw      | 0   
 company        | 0   
 location_raw   | 10  
 post_date      | 0   
 url            | 0   
 type_raw       | 212 
 levels_raw     | 0   
 categories_raw | 0   
 contents       | 0   
 _ingest_ts     | 0   
 _run_label     | 0   
 _source        | 0   



In [0]:
display(bronze_df.show())


+--------+--------------------+--------------------+-----------------+----------+--------------------+--------+------------+------------------+--------------------+-------------------+----------+------------+
|  job_id|           title_raw|             company|     location_raw| post_date|                 url|type_raw|  levels_raw|    categories_raw|            contents|         _ingest_ts|_run_label|     _source|
+--------+--------------------+--------------------+-----------------+----------+--------------------+--------+------------+------------------+--------------------+-------------------+----------+------------+
| 3104241|Merchandise and S...|             Walmart|  Albuquerque, NM|2025-08-13|https://www.themu...|external|   Mid Level|Data and Analytics|<p><b>Position Su...|2025-08-27T23:02:58|    manual|the_muse_api|
| 4278354|Senior Data Scien...|            Bluecore|     New York, NY|2024-11-01|https://www.themu...|external|Senior Level|Data and Analytics|<div>Bluecore is ...|

In [0]:


# Step 1: Transform bronze data
from pyspark.sql.functions import trim, col, regexp_replace, when

# Load Bronze
bronze_df = spark.table("bronze_db.jobs_bronze")

# Transform to Silver
silver_df = (
    bronze_df
    .filter(col("location_raw").isNotNull() & col("type_raw").isNotNull())
    .withColumn("location", trim(col("location_raw")))
    
    # Clean type_raw → remove [ ] and ' characters
    .withColumn("type_raw_clean", regexp_replace(col("type_raw"), r"[\[\]']", ""))
    
    # Replace empty strings with null
    .withColumn("type_cleaned", when(trim(col("type_raw_clean")) == "", None).otherwise(trim(col("type_raw_clean"))))
    
    # Final type column
    .withColumn("type", col("type_cleaned"))

    .drop("type_raw_clean", "type_cleaned")  # optional cleanup
)

# Just before saving to silver table
silver_df = silver_df.withColumn("contents", regexp_replace("contents", "<[^>]*>", ""))



# Save to Silver Table
silver_df.write.mode("overwrite").option("mergeSchema", "true").format("delta").saveAsTable("silver_db.jobs_silver")


# Optional preview
silver_df.show(truncate=False)


+--------+---------------------------------------------------------------------------------------------------------------+---------------------------------+-----------------+----------+--------------------------------------------------------------------------------------------------------------------------------------------+--------+------------+------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
# Check distinct values in type column
silver_df.select("type").distinct().show(truncate=False)



+--------+
|type    |
+--------+
|external|
+--------+



In [0]:
# Inspect raw column to debug
silver_df.select("type_raw").distinct().show(truncate=False)


+--------+
|type_raw|
+--------+
|external|
+--------+



In [0]:
silver_table_name = "jobs_silver"
full_silver_path = f"silver_db.{silver_table_name}"

(
    silver_df
    .write
    .mode("overwrite")        # since we're starting clean
    .format("delta")
    .option("mergeSchema", "true") 
    .saveAsTable(full_silver_path)
)

print(f"✅ Silver table written: {full_silver_path}")

✅ Silver table written: silver_db.jobs_silver


In [0]:
display(spark.sql(f"SELECT * FROM {full_silver_path} LIMIT 10"))
#display(spark.sql("SELECT * FROM bronze_db.jobs_bronze LIMIT 10"))

job_id title_raw company location_raw post_date url type_raw levels_raw categories_raw contents _ingest_ts _run_label _source location type 3104241 Merchandise and Stocking Associate Walmart Albuquerque, NM 2025-08-13 https://www.themuse.com/jobs/walmart/merchandise-and-stocking-associate-68d372 external Mid Level Data and Analytics Position Summary...Do you like to work on your feet and keep things neat and organized? Our merchandising & stocking associates connect all of the dots to make sure members can find everything they have on their shopping list. From every day needs to special occasions, members need you, as their mini tour guide, to take them that special product. Depending on the shift you work, your job could include moving inventory in the backroom, unloading trucks, fulfilling club pick-up orders or helping members while stocking shelves. From unloading trucks in the summer to filling ice cream in the freezer, this fast-paced job can be physically demanding - it's like being paid to go to the gym! Be a part of a great team with a common goal - making sure members can find more of what they love, for less. You will sweep us off our feet if:• You thrive in fast-paced environments • You keep member satisfaction as your top priority• You're comfortable with change and quickly adapt to different work scenarios• You're a problem solver who tackles obstacles head-on to ensure each task is completed with excellence• You are able to pick up boxes and other heavy objects weighing more than 25 poundsYou will make an impact by:• Promptly unloading trucks • Assisting fellow associates as needed throughout the store• Sorting and stocking products on shelves and in the backroom• Engaging with vendors and drivers with a positive attitude• Maintaining a clean, neat, and member-ready areaThe merchandising & stocking associate role is a great way to start a fulfilling career at Sam's Club. Apply now!The above information has been designed to indicate the general nature and level of work performed in the role. It is not designed to contain or be interpreted as a comprehensive inventory of all duties, responsibilities and qualifications required of employees assigned to this job. The full job description can be made available as part of the hiring process.What you'll do...Provides member service by acknowledging the member and identifying member needs; providing guidance and support to members regarding self- service technology; assisting members with purchasing decisions; locating merchandise; resolving member issues and concerns; and promoting the company's products and services.Maintains safety of facility according to company policies and procedures by conducting safety sweeps; following forklift spotting procedures; following procedures for handling and disposing of hazardous materials; following company steel standard guidelines; and correcting and reporting unsafe situations to management.Maintains the sales floor and merchandise presentation in accordance with company policies and procedures by properly zoning the area; stocking, arranging and organizing merchandise; setting up, cleaning, and organizing product displays; removing damaged goods; signing and pricing merchandise according to company policies and procedures; identifying shrink and damages; and securing fragile and high-shrink merchandise.Monitors food and merchandise quality by ensuring product rotation, code dating, product recalls, and sanitation standards are followed according to company policies and procedures; and ensuring prominent display of promotional and seasonal merchandise.Complies with company policies, procedures, and standards of ethics and integrity by implementing related action plans; using the Open Door Policy; and applying these in executing business processes and practices.Completes work assignments and priorities by using policies, data, and resources; collaborating with managers, co-workers, customers, and other business partners; identif